In [ ]:
%matplotlib inline



# Connect workflows on different processes implicitly
This example shows how distributed files can be read and post processed
on distributed processes. After remote post processing,
results a merged on the local process. In this example, different workflows are
directly created on different servers. Those workflows are then connected
together without having to care that they are on remote processes.


Import dpf module and its examples files



In [ ]:
from ansys.dpf import core as dpf
from ansys.dpf.core import examples
from ansys.dpf.core import operators as ops

# Configure the servers
Start three dpf servers.



In [ ]:
server1 = dpf.start_local_server(as_global=False)
server2 = dpf.start_local_server(as_global=False)
server3 = dpf.start_local_server(as_global=False)

# Create template workflows on remote servers
For the purpose of this example, we will create 2 workflows computing
elemental nodal stresses on different servers. The second workflow will
multiply by 2.0 the stresses. A last workflow will merge the outputs



In [ ]:
local_files = examples.download_distributed_files()

first workflow S



In [ ]:
workflow1 = dpf.Workflow(server=server1)
server_file_path1 = dpf.upload_file_in_tmp_folder(local_files[0], server=server1)
model = dpf.Model(server_file_path1, server=server1)
stress1 = model.results.stress()
workflow1.add_operator(stress1)
workflow1.set_output_name("out1", stress1.outputs.fields_container)

second workflow S*2.0



In [ ]:
workflow2 = dpf.Workflow(server=server2)
server_file_path2 = dpf.upload_file_in_tmp_folder(local_files[1], server=server2)
model = dpf.Model(server_file_path2, server=server2)
stress2 = model.results.stress()
mul = stress2 * 2.0
workflow2.add_operator(mul)
workflow2.set_output_name("out2", mul.outputs.fields_container)

third workflow merge



In [ ]:
workflow3 = dpf.Workflow(server=server3)
merge = ops.utility.merge_fields_containers()
nodal = ops.averaging.to_nodal_fc(merge)
workflow3.add_operators([merge, nodal])
workflow3.set_input_name("in1", merge, 0)
workflow3.set_input_name("in2", merge, 1)
workflow3.set_output_name("merged", nodal.outputs.fields_container)

# Connect the workflows together and get the output



In [ ]:
workflow3.connect_with(workflow1, ("out1", "in1"))
workflow3.connect_with(workflow2, ("out2", "in2"))

fc = workflow3.get_output("merged", dpf.types.fields_container)
fc[0].meshed_region.plot(fc[0])